In [ ]:
#This script creates a binary spreadsheet of all participants enrolled in EF and whether or not their 
#pre/post scan SR data is complete 
#Inputs:
    #enrollment sheets pulled from AXIS, stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs
    #all diagnosis data from LIBI, pulled from oracle and stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs
#Outputs:
    #merged sheet of all EF diagnoses, stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/outputs
    #a binary spreadsheet of diagnosis collected for T1 participants under LIBI enrolled prior to April 1st, 2022 


In [1]:
import pandas as pd

In [2]:
#read in T1 enrollment
axis_t1=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t1.csv',dtype=str)
axis_t1=axis_t1.drop(columns=['scan_1_date'])

In [3]:
#reformat 
t1_enroll=axis_t1['bblid']
t1_enroll=t1_enroll.tolist()
t1_enroll = [str(t) for t in t1_enroll]

In [9]:
#read in all LiBI dx data 
libi_dx=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/oracle_dx_libi.csv')
libi_dx=libi_dx.drop(columns=['DODIAGNOSIS'])
libi_dx=libi_dx.astype(str)

In [10]:
#filter for EF participants
ef_dx=libi_dx[libi_dx['BBLID'].isin(t1_enroll)]
ef_dx.head(n=5)

,BBLID,CONSENSUS_TYPE,DOCONSENSUS,CONSENSUSBY,INTERVIEWER,PROTOCOL,VISITNUM,TYPE,INTERVIEWEE,HSTATUS,...,AGEONSET_AXIS1,AXIS1_LIFETIME,AXIS1_UK,AGEONSET_CLINICRISK,CLINICRISK_LIFETIME,CLINICRISK_UK,DEFICIT,ENTBY,DOENT,Unnamed: 92
48,19861,Faculty Review,08/23/18,Ran,rsga,16-013305 - LIBI_Common,1.0,P,proband,BP1,...,nan,nan,nan,nan,nan,nan,N,rsga,00:00.0,nan
69,20124,Faculty Review,06/25/19,Monica,ehaderer,16-013305 - LIBI_Common,nan,P,proband,NC,...,nan,nan,nan,nan,nan,nan,nan,ehaderer,00:00.0,nan
70,20125,Case Conference,12/20/18,"Monica, Ran, Jerome",diegodav,16-013305 - LIBI_Common,nan,P,proband,PRO,...,nan,nan,nan,nan,nan,nan,nan,diegodav,00:00.0,nan
72,20139,Faculty Review,10/03/19,Monica,kpiiwaa,16-013305 - LIBI_Common,nan,P,both,O,...,nan,nan,nan,nan,nan,nan,nan,redcap,00:00.0,nan
73,20141,Faculty Review,09/16/19,Lauren,rsga,16-013305 - LIBI_Common,nan,P,proband,NC,...,nan,nan,nan,nan,nan,nan,nan,redcap,00:00.0,nan


In [11]:
#loop through and make a list of all enrolled ID's that are are in the data 
has_dx=[]
for index, row in ef_dx.iterrows():
    for t in t1_enroll:
        if (t in row['BBLID']):
            has_dx.append(t)

In [12]:
audit = pd.DataFrame (t1_enroll, columns = ['BBLID'])
print(len(audit))

173


In [13]:
#check if each ID is in the list
dx=[]
for val in audit.values:
    if val in has_dx:
        dx.append(1)
    if val not in has_dx:
        dx.append(0)

In [14]:
#add column
audit['diagnosis']=dx

In [17]:
audit.head(n=5)

,BBLID,diagnosis
0,19861,1
1,20124,1
2,20125,1
3,20139,1
4,20141,1


In [16]:
audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_dx_audit.csv', sep = ',', index=False)

In [15]:
ef_dx.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/outputs/EF_all_dx.csv', sep = ',', index=False)